In [2]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['용어 설명']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    term_set = dict()
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data_lines = source_data.open()
        for line in source_data_lines.readlines():
            line = json.loads(line)
            if 'term' in line:
                term = line['term']
                term_dict = dict()
                term_dict['id'] = line['id']
                sense_no = line['sense_no']
                term_dict['facet'] = line['facet']
                term_dict['domain'] = [line['top_level_domain'],line['level2'],line['level3']]
                term_dict['definition'] = line['definition']
                if term in term_set:
                    term_set[term][sense_no] = term_dict
                else:
                    term_set[term] = {sense_no: term_dict}
                    
        source_data_lines = source_data.open()
        for line in source_data_lines.readlines():
            line = json.loads(line)
            sentence = line.get('sentence')
            if sentence:
                for term in line['tokens']:
                    token = sentence[term['start']:term['start']+term['length']]
                    if token in term_set:
                        term_desc = term_set[token][term['sense_no']]
                            
                        #### 용어 설명
                        data = {'input': {}, 'output': None}
                        ## preprocess data from line
                        data['input']['sentence'] = sentence
                        data['input']['term'] = token
                        data['output'] = term_desc
                        ## preprocess data from line end
                        task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
                        # print(json.dumps(data, indent=4, ensure_ascii=False))
                        #### 용어 설명 end
                    else:
                        print(line)
            
            
            # break
        # break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 1/1 [00:02<00:00,  2.82s/it]
